In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.manifold import TSNE
from gensim.test.utils import common_texts # pip install gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [52]:
# Python doc2vec model 학습 및 저장
# 가변 길이의 콘텐츠를 고정 길이의 콘텐츠로 -> 가변 길이의 정책을 128차원의 고정 길이의 벡터로
# 매개변수 tok: boolean type으로 형태소 분석기를 적용한 모델인가 아닌가를 의미

def make_doc2vec_models(tagged_data, tok, vector_size=128, window = 3, epochs = 40, min_count = 0, workers = 4):
    model = Doc2Vec(tagged_data, vector_size=vector_size, window=window, epochs=epochs, min_count=min_count, workers=workers)
    model.save(f'./{tok}_news_model.doc2vec')

In [53]:
# 데이터 전처리
def get_preprocessing_data(data):
    data.drop(['대상', '접수시작', '접수종료', 'URL', '번호'], axis = 1, inplace =True)
    
    category_mapping = {
        'youth_support' : 0,       # 청년 지원
        'youth_job' : 1,           # 청년 취업
        'birth_infant' : 2,        # 출산 및 육아
        'marriage_family' : 3,     # 결혼 및 가정
        'job_found' : 4,           # 취업 및 창업
        'subsidy_supportFund' : 5,   # 보조금 및 지원금
        'learning' : 6,            # 학업
        'agriculture' : 7,         # 농업
        'old' : 8,                 # 노인
        'disability' : 9,          # 장애
    }

    data['해시태그'] = data['해시태그'].map(category_mapping)
    data['title_content'] = data['사업명'] + " " + data['기타'] + " " + data['지자체']
    data.drop(['사업명', '기타', '지자체'], axis = 1, inplace = True)
    return data

In [54]:
# doc2vec에 필요한 데이터 만들기
def make_doc2vec_data(data, column, t_document=False):
    data_doc = []
    for tag, doc in zip(data.index, data[column]):
        doc = doc.split(" ")
        data_doc.append(([tag], doc))
    if t_document:
        data = [TaggedDocument(words=text, tags=tag) for tag, text in data_doc]
        return data
    else:
        return data_doc

In [55]:
# 코사인 유사도를 구해 상위 n개를 추천
def get_recommened_contents(user, data_doc, model):
    scores = []

    for tags, text in data_doc:
        trained_doc_vec = model.docvecs[tags[0]]
        scores.append(cosine_similarity(user.reshape(-1, 128), trained_doc_vec.reshape(-1, 128)))

    scores = np.array(scores).reshape(-1)
    scores = np.argsort(-scores)[:5]
    
    return data.loc[scores, :]

In [56]:
# content vector 기반의 user history를 평균하여 user embedding 만들기
def make_user_embedding(index_list, data_doc, model):
    user = []
    user_embedding = []
    for i in index_list:
        user.append(data_doc[i][0][0])
    for i in user:
        user_embedding.append(model.docvecs[i])
    user_embedding = np.array(user_embedding)
    user = np.mean(user_embedding, axis = 0)
    return user

In [57]:
# 사용자 히스토리 확인
def view_user_history(data):
    print(data[['category', 'title_content']])

In [117]:
# 키워드 추출, 빈보둔석 코드

# 모듈 import
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
from collections import Counter
from pykospacing import Spacing #pip install git+https://github.com/haven-jeon/PyKoSpacing.git 
import time

# 빈도분석 + 키워드 추출 함수

def load_stopwords():
    with open('stopwords.txt', 'r') as f:
        list_file = f.readlines()
    return list_file[0].split(",")

# 정규 표현식을 통해 한글 단어만 남기기 (이모티콘, 초성, 영어 제거)
def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text)
    return result

def make_wordlist(body,stopwords):
    result = []
    for i in body:
        #정규표현식 적용
        #print("데이터 정제 중....")
        words = pd.Series(i)
        words = words.apply(lambda x:extract_word(x))
        spacing = Spacing()
        words = words.apply(lambda x:spacing(x))
        
        #형태소 추출
        #print("형태소 추출 중....")
        okt = Okt()
        words = " ".join(words.tolist())
        words = okt.morphs(words,stem=True)
        
        #한글자 제거
        #print("한글자 제거 중....")
        words = [x for x in words if len(x)>1]
        
        #불용어 제거
        #print("불용어 제거 중....")
        words = [x for x in words if x not in stopwords]
        
        # 형태소 분석 결과를 리스트에 저장
        result.append(words)
        
    return result

'\ndef run():\n    body = \n    print(make_wordlist(body, load_stopwords()))\n'

In [77]:
data = pd.read_csv('해커톤.csv', encoding='ms949')
data.head()

,번호,사업명,대상,접수시작,접수종료,지자체,기타,해시태그,URL
0,1,2022년 전기자동차 보급사업(하반기) 시행 공고,"개인, 개인사업자, 법인, 외국인",2022.07.04,예산 소진시,경산시,"전기차, 보조금, 전기자동차",subsidy_supportFund,https://www.gbgs.go.kr/open_content/ko/page.do...
1,2,1인 미디어 콘텐츠산업 육성 사업,개인,2022,2023,경산시,"4차산업혁명, 크리에이터, 유튜브, 미디어",job_found,https://www.gbgs.go.kr/open_content/ko/page.do...
2,3,경북 청년 키친랩 조성 및 운영,"개인, 팀",2018,2022,경산시,"외식업, 창업, 요리, 주방",youth_support,https://www.gbgs.go.kr/open_content/ko/page.do...
3,4,2022년 주산지일관 기계화 농기계 지원사업 신청자 모집 공고,농업단체,2022.09.23,2022.09.30,경산시,농기계,agriculture,https://www.gbgs.go.kr/open_content/ko/page.do...
4,5,경로당운영 지원,경로당,NaN,NaN,경산시,경로당,old,https://www.gbgs.go.kr/special/silver/page.do?...


In [78]:
# 데이터 전처리
get_preprocessing_data(data)

,해시태그,title_content
0,5,"2022년 전기자동차 보급사업(하반기) 시행 공고 전기차, 보조금, 전기자동차 경산시"
1,4,"1인 미디어 콘텐츠산업 육성 사업 4차산업혁명, 크리에이터, 유튜브, 미디어 경산시"
2,0,"경북 청년 키친랩 조성 및 운영 외식업, 창업, 요리, 주방 경산시"
3,7,2022년 주산지일관 기계화 농기계 지원사업 신청자 모집 공고 농기계 경산시
4,8,경로당운영 지원 경로당 경산시
5,8,"노인일자리 및 사회활동지원사업 노인, 일자리, 사회참여, 노후소득 경산시"
6,9,"장애인휠체어 등 수리비용 지원 장애인, 휠체어 경산시"
7,1,"청년취업날개 코디네이터 면접, 양복, 양복대여, 정장 경산시"
8,2,"임산부 건강관리 지원 임산부 건강, 임산부 검진 경산시"
9,3,"신혼부부 임차보증금 이자 지원 임차보증금, 이자지원, 대출 경산시"


In [132]:
data['mecab_tok'] = make_wordlist(data['title_content'], load_stopwords())

In [141]:
no = 0
for i in data['mecab_tok']:
    str_list = i
    result = ' '.join(s for s in str_list)
    data['mecab_tok'][no] = result
    no += 1

data

C:\Users\User\AppData\Local\Temp/ipykernel_6896/1068817883.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mecab_tok'][no] = result


,해시태그,title_content,mecab_tok
0,5,"2022년 전기자동차 보급사업(하반기) 시행 공고 전기차, 보조금, 전기자동차 경산시",전기자동차 보급 사업 하반기 시행 공고 전기차 보조금 전기자동차 경산시
1,4,"1인 미디어 콘텐츠산업 육성 사업 4차산업혁명, 크리에이터, 유튜브, 미디어 경산시",미디어 콘텐츠 산업 육성 사업 산업혁명 크리에이터 유튜브 미디어 경산시
2,0,"경북 청년 키친랩 조성 및 운영 외식업, 창업, 요리, 주방 경산시",경북 청년 키친 조성 운영 외식 창업 요리 주방 경산시
3,7,2022년 주산지일관 기계화 농기계 지원사업 신청자 모집 공고 농기계 경산시,주산지 일관 기계화 농기 지원 사업 신청 모집 공고 농기 경산시
4,8,경로당운영 지원 경로당 경산시,경로당 운영 지원 경로당 경산시
5,8,"노인일자리 및 사회활동지원사업 노인, 일자리, 사회참여, 노후소득 경산시",노인 일자리 사회 활동 지원 사업 노인 일자리 사회 참여 노후 소득 경산시
6,9,"장애인휠체어 등 수리비용 지원 장애인, 휠체어 경산시",장애인 휠체어 수리 비용 지원 장애인 휠체어 경산시
7,1,"청년취업날개 코디네이터 면접, 양복, 양복대여, 정장 경산시",청년 취업 날개 디네 면접 양복 양복 대여 정장 경산시
8,2,"임산부 건강관리 지원 임산부 건강, 임산부 검진 경산시",임산부 건강 관리 지원 임산부 건강 임산부 검진 경산시
9,3,"신혼부부 임차보증금 이자 지원 임차보증금, 이자지원, 대출 경산시",신혼부부 임차보증금 이자 지원 임차보증금 이자 지원 대출 경산시


In [147]:
data.rename(columns = {'해시태그' : 'category'}, inplace=True)

In [148]:
data

,category,title_content,mecab_tok
0,5,"2022년 전기자동차 보급사업(하반기) 시행 공고 전기차, 보조금, 전기자동차 경산시",전기자동차 보급 사업 하반기 시행 공고 전기차 보조금 전기자동차 경산시
1,4,"1인 미디어 콘텐츠산업 육성 사업 4차산업혁명, 크리에이터, 유튜브, 미디어 경산시",미디어 콘텐츠 산업 육성 사업 산업혁명 크리에이터 유튜브 미디어 경산시
2,0,"경북 청년 키친랩 조성 및 운영 외식업, 창업, 요리, 주방 경산시",경북 청년 키친 조성 운영 외식 창업 요리 주방 경산시
3,7,2022년 주산지일관 기계화 농기계 지원사업 신청자 모집 공고 농기계 경산시,주산지 일관 기계화 농기 지원 사업 신청 모집 공고 농기 경산시
4,8,경로당운영 지원 경로당 경산시,경로당 운영 지원 경로당 경산시
5,8,"노인일자리 및 사회활동지원사업 노인, 일자리, 사회참여, 노후소득 경산시",노인 일자리 사회 활동 지원 사업 노인 일자리 사회 참여 노후 소득 경산시
6,9,"장애인휠체어 등 수리비용 지원 장애인, 휠체어 경산시",장애인 휠체어 수리 비용 지원 장애인 휠체어 경산시
7,1,"청년취업날개 코디네이터 면접, 양복, 양복대여, 정장 경산시",청년 취업 날개 디네 면접 양복 양복 대여 정장 경산시
8,2,"임산부 건강관리 지원 임산부 건강, 임산부 검진 경산시",임산부 건강 관리 지원 임산부 건강 임산부 검진 경산시
9,3,"신혼부부 임차보증금 이자 지원 임차보증금, 이자지원, 대출 경산시",신혼부부 임차보증금 이자 지원 임차보증금 이자 지원 대출 경산시


In [142]:
# 전처리한 데이터를 활용하여 doc2vec model 만들기
# doc2vec에 활용될 수 있도록 데이터 전처리하기 -> make_doc2vec_data 함수
# _tag 데이터는 doc2vec model 학습에 사용
# _tag 없는 데이터는 user embedding, cosine similarity를 구할 때 사용
data_doc_title_content_tag = make_doc2vec_data(data, 'title_content', t_document=True)
data_doc_title_content = make_doc2vec_data(data, 'title_content')
data_doc_tok_tag = make_doc2vec_data(data, 'mecab_tok', t_document=True)
data_doc_tok = make_doc2vec_data(data, 'mecab_tok')

In [143]:
# doc2vec model 학습 -> make_doc2vec_models 함수 사용
make_doc2vec_models(data_doc_title_content_tag, tok=False)

In [144]:
# doc2vec model 학습 -> make_doc2vec_models 함수 사용
make_doc2vec_models(data_doc_tok_tag, tok=True)

In [149]:
# 임시로 사용자 히스토리 만들기
user_category_1 = data.loc[random.sample(data.loc[data.category == 0, :].index.values.tolist(), 5), :]  #청년 지원
view_user_history(user_category_1)

    category                                   title_content
15         0  중소기업 청년근로자 3,6,9 미래희망 지원사업 중소기업, 근속장려금, 취업 김천시
28         0                    청년매입임대주택사업 임대주택, 청년주택, 주거 대구
2          0           경북 청년 키친랩 조성 및 운영 외식업, 창업, 요리, 주방 경산시
26         0   대구 청년사회진입 활동지원금 활동지원금, 지원금, 청년, 상담연결, 진로탐색 대구
18         0     춘천시 전입대학생 전입장려금 주거,금융,주거지원,재학생, 대학교, 전입 춘천시


In [150]:
user_category_2 = data.loc[random.sample(data.loc[data.category == 1, :].index.values.tolist(), 5), :]  #청년 취업
view_user_history(user_category_2)

    category                                  title_content
20         1                청년 취업수당 지원 취업지원, 교육훈련, 취업수당 고성군
31         1                         월급받는 청년농부제 청년, 농업 경상북도
7          1              청년취업날개 코디네이터 면접, 양복, 양복대여, 정장 경산시
19         1       청년 취업지망생 구직활동지원사업 구직, 졸업, 취업지원, 교육훈련 양구군
29         1  대구 희망옷장(대학생 취업패키지 지원사업) 면접정장, 무료대여, 정장, 면접 대구


In [152]:
# 모델 불러오기
model_title_content = Doc2Vec.load('./False_news_model.doc2vec')
model_tok = Doc2Vec.load('./True_news_model.doc2vec')

In [153]:
# 사용자 히스토리를 바탕으로 user embedding
# make_user_embedding 함수에 각 사용자가 본 정책의 index list를 전달
user_1 = make_user_embedding(user_category_1.index.values.tolist(), data_doc_title_content, model_title_content) # 청년 지원
user_2 = make_user_embedding(user_category_2.index.values.tolist(), data_doc_title_content, model_title_content) # 청년 취업

C:\Users\User\AppData\Local\Temp/ipykernel_6896/2953635978.py:8: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  user_embedding.append(model.docvecs[i])


In [154]:
# user_1 추천 받기
# user embedding 결과와 doc2vec data를 get_recommend_contents 함수에 전달
result = get_recommened_contents(user_1, data_doc_title_content, model_title_content)
pd.DataFrame(result.loc[:, ['category', 'title_content']])

C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec 

,category,title_content
26,0,"대구 청년사회진입 활동지원금 활동지원금, 지원금, 청년, 상담연결, 진로탐색 대구"
2,0,"경북 청년 키친랩 조성 및 운영 외식업, 창업, 요리, 주방 경산시"
15,0,"중소기업 청년근로자 3,6,9 미래희망 지원사업 중소기업, 근속장려금, 취업 김천시"
18,0,"춘천시 전입대학생 전입장려금 주거,금융,주거지원,재학생, 대학교, 전입 춘천시"
28,0,"청년매입임대주택사업 임대주택, 청년주택, 주거 대구"


In [160]:
# user_2 추천받기
result = get_recommened_contents(user_2, data_doc_title_content, model_title_content)
pd.DataFrame(result.loc[:, ['category', 'title_content']])

C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec 

,category,title_content
31,1,"월급받는 청년농부제 청년, 농업 경상북도"
29,1,"대구 희망옷장(대학생 취업패키지 지원사업) 면접정장, 무료대여, 정장, 면접 대구"
20,1,"청년 취업수당 지원 취업지원, 교육훈련, 취업수당 고성군"
19,1,"청년 취업지망생 구직활동지원사업 구직, 졸업, 취업지원, 교육훈련 양구군"
7,1,"청년취업날개 코디네이터 면접, 양복, 양복대여, 정장 경산시"


## 형태소 분석 후 결과

In [161]:
# 사용자 히스토리
user_1 = make_user_embedding(user_category_1.index.values.tolist(), data_doc_tok, model_tok) 
user_2 = make_user_embedding(user_category_2.index.values.tolist(), data_doc_tok, model_tok) 

C:\Users\User\AppData\Local\Temp/ipykernel_6896/2953635978.py:8: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  user_embedding.append(model.docvecs[i])


In [162]:
# 추천 받기
result = get_recommened_contents(user_1, data_doc_tok, model_tok)
pd.DataFrame(result.loc[:, ['category', 'title_content']])

C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec 

,category,title_content
26,0,"대구 청년사회진입 활동지원금 활동지원금, 지원금, 청년, 상담연결, 진로탐색 대구"
18,0,"춘천시 전입대학생 전입장려금 주거,금융,주거지원,재학생, 대학교, 전입 춘천시"
2,0,"경북 청년 키친랩 조성 및 운영 외식업, 창업, 요리, 주방 경산시"
25,6,"북한이탈청소년 교육프로그램 지원 탈북, 북한, 청소년, 사회적응 춘천시"
0,5,"2022년 전기자동차 보급사업(하반기) 시행 공고 전기차, 보조금, 전기자동차 경산시"


In [163]:
# 추천 받기
result = get_recommened_contents(user_2, data_doc_tok, model_tok)
pd.DataFrame(result.loc[:, ['category', 'title_content']])

C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec = model.docvecs[tags[0]]
C:\Users\User\AppData\Local\Temp/ipykernel_6896/957220760.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  trained_doc_vec 

,category,title_content
29,1,"대구 희망옷장(대학생 취업패키지 지원사업) 면접정장, 무료대여, 정장, 면접 대구"
25,6,"북한이탈청소년 교육프로그램 지원 탈북, 북한, 청소년, 사회적응 춘천시"
31,1,"월급받는 청년농부제 청년, 농업 경상북도"
19,1,"청년 취업지망생 구직활동지원사업 구직, 졸업, 취업지원, 교육훈련 양구군"
26,0,"대구 청년사회진입 활동지원금 활동지원금, 지원금, 청년, 상담연결, 진로탐색 대구"


### 형태소 분석 후 결과는 성능이 썩 좋지 않음.